# CAP Talks

Разберем на примере генерации текста работу с Jupyter Notebook, нейронными сетями и сопутствующими библиотеками.

Анализ текста является довольно сложной задачей для решения посредством обычного кода: огромное количество слов, зависимостей между ними не позволят создать хорошо расширяемый и общий алгоритм. Однако, для нейронной сети эта задача одна из самых типичных.

## Что будем "читать"?
В качестве вводных данных для сети будем использовать несколько произведений Говарда Лавкрафта. Его стиль отличается малым количеством диалогов, зачастую написан в одном стиле, жанре, а значит будет проще научить сеть создавать текст, который хотя бы издалека будет напоминать авторский.

- **Тень над Иннсмутом**
- **Мифы Ктулху**
- **Безымянный город** и другие

![Lovecraft](img/lovecraft.jpg)

### Почему не на русском?
Английский язык гораздо проще для анализа, а следовательно и сгенерированный текст будет обладать меньшим количеством ошибок.

## Keras — открытая нейросетевая библиотека, написанная на языке Python
- Open-source
- Работает поверх TensorFlow, Microsoft Cognitive Toolkit, Theano, or PlaidML
- Нацелена на оперативную работу с сетями глубинного обучения
- Спроектирована так, чтобы быть компактной, модульной и расширяемой

## Жизненный цикл Keras
![Keras Lifecycle](img/lifecycle.png)

In [ ]:
from __future__ import print_function
from keras.models import Sequential, Model, load_model
from keras.layers import Dense, Activation, Dropout
from keras.layers import LSTM, Input, Flatten, Bidirectional
from keras.layers.normalization import BatchNormalization
from keras.optimizers import Adam
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.metrics import categorical_accuracy
from keras.utils import Sequence
import tensorflow as tf
import numpy as np
import random
import sys
import os
import time
import spacy
import codecs
import collections
from six.moves import cPickle

print('Ready!')

# Чтение входного текста

Необходимо максимально облегчить задачу нейронной сети: необходимо избавиться от ненужных символов и привести текст к нижнему регистру. Таким образом мы снизим размер словаря.

In [ ]:
data_dir = '..\Data'
save_dir = '..\Save'
input_file = os.path.join(data_dir, 'lovecraft.txt')

# reading the file
with codecs.open(input_file, "r") as f:
    data = f.read()
    
# creating words list
nlp = spacy.load('en_core_web_sm') # spacy lib and English model to process the text
doc = nlp(data)

wordlist = []
for word in doc:
    if word.text not in ("\n","\n\n",'\u2009','\xa0'):
        wordlist.append(word.text.lower())

print('First 10 words: ', wordlist[:10])
print('Total words: ', len(wordlist))

# Составление словаря

Словарь будет содержать список слов и их индексы.

In [ ]:
word_counts = collections.Counter(wordlist)

# map words on indexes
vocabulary_list = list(sorted(set(wordlist)))
vocab = {x: i for i, x in enumerate(vocabulary_list)}

vocab_size = len(vocab)
    
print('Vocabulary size: ', vocab_size)
print('20 most common words: ', word_counts.most_common()[:20])

# Создание списка последовательностей для обучения сети
Нам необходимо 2 списка:
 - **sequences**: последовательности слов, которые будем использовать для обучения сети
 - **next_words**: список следующих слов, основанных на **sequences**
 
30 первых слов из нашего текста будет первой последовательностью. 31 слово заносим в список "следующих", **next_words**.
Далее, переходя на одно слово вперед по тексту, мы создадим следующую последовательность, и так до конца текста.

![Sequences](img/sequences.png)

In [ ]:
sequences_length = 30
next_words = []
sequences = []

for i in range(0, len(wordlist) - sequences_length, 1):
    sequences.append(wordlist[i: i + sequences_length])
    next_words.append(wordlist[i + sequences_length])

print('Sequences total: ', len(sequences))
print('First sequence: ', sequences[0])
print('First sequence next word: ', next_words[0])
print('Ready!')

Так как сеть работает с числовыми значениями, необходимо перевести каждое слово на соответствующее ему значение индекса и провести векторизацию.

Получим следующие матрицы:
 - X : матрица векторов 'последовательностей' со следующими измерениями:
     - массив последовательностей,
     - массив слов в последовательности,
     - словарь для каждого слова.
 - Y : матрица векторов 'следующих' слов со следующими измерениями:
     - массив последовательностей,
     - словарь для каждого слова.
     
Для каждого слова мы определяем его индекс в словаре и устанавливаем True в матрице на его позиции.

In [ ]:
# vectorization
X = np.zeros((len(sequences), sequences_length, vocab_size), dtype = np.bool)
Y = np.zeros((len(sequences), vocab_size), dtype = np.bool)
for i, sequence in enumerate(sequences):
    for t, word in enumerate(sequence):
        X[i, t, vocab[word]] = 1
    Y[i, vocab[next_words[i]]] = 1

print('X consists of', len(sequences), 'sequences, each sequence contains', len(X[0]), 'words')
print('Y consists of', len(Y), '"next" words')
print('Each word represented as True in vocabulary of', len(X[0][0]), 'elements')

print('Ready!')

# Выбор типа нейронной сети

Для начала необходимо выбрать тип нейронной сети.

Нейронные сети прямого распространения (feed-forward) не содержат петель, то есть информация передается по сети только вперед, не имея возможности повлиять на ввод.

![Feed-forward neural network](img/fnn.png)
<h3 align="center"><i>Feed-forward neural network</i></h3>

Такая сеть нам не подходит поскольку мы будем иметь дело с данными, имеющими характерную последовательность - предложения, а значит хотелось бы иметь возможность "запоминать" выходные значения нейронов и влиять ими на входные данные.

Такие сети называются **рекуррентными**. Они имеют нейроны, которые срабатывают в течение некоторого ограниченного периода времени, прежде чем перейти в состояние покоя. Это срабатывание может стимулировать другие нейроны, которые могут срабатывать немного позже, также в течение ограниченного времени. Это вызывает еще большее количество нейронов, и поэтому со временем мы получаем каскад нейронов, влияющих друг на друга.

![Recurrent neural network](img/rnn.png)
<h3 align="center"><i>Recurrent neural network</i></h3>

### Выбор рекуррентной сети

Мы будем использовать архитектурную разновидность рекуррентной нейронной сети - **Bidirectional Long Short-Term Memory, LSTM**. Архитектура двунаправленной долгой краткосрочной памяти отличается более быстрой сходимостью по сравнению с однонаправленной, а также показала большую точность в тестах.

**LSTM** обычно используют в аналитических задачах на прогнозирование временных рядов. Прогнозирование отлично подходит под описание нашей задачи - сеть должна выбирать наиболее вероятное слово, основываясь на последовательности слов, заданной пользователем.

# Создание нейронной сети

Наша сеть будет иметь следующие характеристики и архитектуру:

 - последовательная архитектура определяет общую конструкцию сети - линейн
 - архитектура bidirectional LSTM размером 256 с функцией активации ReLU
 - dropout с rate 0,6 (помогает предотвратить переобучение (overfitting)) 
 - классический полносвязный слой размером в количество слов в словаре с активацией Softmax для вывода вероятности слова
 - оптимизатор ADAM
 
 ![Network architecture](img/nn.png)
<h4 align="center"><i>Network architecture</i></h4>

  ![Dropout](img/dropout.png)
<h4 align="center"><i>Dropout</i></h4>

In [ ]:
model = Sequential()
model.add(Bidirectional(LSTM(256, activation = "relu"), 
                        input_shape = (sequences_length, vocab_size)))
model.add(Dropout(0.6))
model.add(Dense(vocab_size))
model.add(Activation('softmax'))

model.compile(loss = 'categorical_crossentropy',
              optimizer = Adam(lr = 0.001))
model.summary()

# Обучение нейронной сети

- Тренируем 70 эпох: проход по ВСЕМ тренировочным данным + backprop (epochs = 70)
- Сохраняем сеть каждые 10 эпох в отдельный файл (period = 10, ~230 MB каждый)
- Обучаем на 128 сэмплах (последовательностях слов) за одну итерацию, после чего обновляем градиент (batch_size = 128)
- Каждую эпоху перемешиваем тренировочные данные (shuffle = True)
- Обучаем сеть на 90% тренировочных данных и 10% тестовых (validation_split = 0.1)

In [ ]:
callbacks=[ModelCheckpoint(
               filepath = save_dir + "/" + 'lovecraft-{epoch:02d}.hdf5',
               verbose = 0,
               mode = 'auto',
               period = 10)]

history = model.fit(
             X[:500], # remove [:500] to use whole dataset
             Y[:500], # remove [:500] to use whole dataset
             batch_size = 128,
             shuffle = True,
             epochs = 1, # 70
             callbacks = callbacks,
             validation_split = 0.1)

print('Ready!')

In [ ]:
model.save(save_dir + "/" + 'lovecraft-final.hdf5')

print('Model saved!')

# Генерация текста

Обучение закончено, мы готовы генерировать текст. Для этого необходимо:

- создать текст длиной 30 слов (требуется сетью для последовательностей)
- создать прогноз на следующее слово
- отбросить первое слово и добавить спрогнозированное
- повторить N раз

In [ ]:
seed_sentences = ("cthulhu is the most frightening thing in the world . "
                  "one day the cult will find the hideous citadel of terrifying creature "
                  "and emerge it from the dark sea . ")

print('Generating text with the following seed: "' + seed_sentences + '"')

In [ ]:
def generateText(input_sentences, words_number):
    generated = input_sentences
    sentences_splitted = input_sentences.split()
    
    # generate the text
    for i in range(words_number):
        
        # vectorize the input
        x = np.zeros((1, sequences_length, vocab_size))
        for t, word in enumerate(sentences_splitted):
            x[0, t, vocab[word]] = 1.
    
        # calculate next word
        preds = model.predict(x, verbose=0)[0]
        next_index = np.argmax(preds)
        next_word = vocabulary_list[next_index]
    
        # add the next word to the text
        generated += " " + next_word
        
        # shift the sentence by one, add the next word at its end
        sentences_splitted = sentences_splitted[1:] + [next_word]
    return generated

In [ ]:
def prettify(generated):
    punctuation = [' ', ',', '.', '!', '?', ';', '\'']
    pretty = ''
    
    for i, ltr in enumerate(generated):
        if i == 0 or generated[i - 2] == '.':
            pretty += ltr.upper()
            continue
        if ltr == ' ' and generated[i + 1] in punctuation:
            continue
        pretty += ltr
    return pretty

In [ ]:
generated = generateText(seed_sentences, 100)
prettyfied = prettify(generated)

print(prettyfied)

In [ ]:
model = load_model(save_dir + "/" + 'lovecraft_70epochs.hdf5')

print('Loaded!')

In [ ]:
generated = generateText(seed_sentences, 100)
prettyfied = prettify(generated)

for i, sentence in enumerate(prettyfied.split('. ')):
    print(sentence + '.')

In [ ]:
![Meme](img/meme.jpg)